# Industrial Safety Equipment Detection - Training Notebook

This notebook trains a YOLOv8 model on the synthetic dataset from Duality AI's Falcon platform to detect FireExtinguisher, ToolBox, and OxygenTank objects.

## Features:
- YOLOv8 architecture with optimized parameters
- Enhanced augmentation pipeline
- Comprehensive training monitoring
- Industrial safety equipment detection (3 classes)
- Synthetic dataset from Duality AI's Falcon platform

## 1. Setup and Imports

In [ ]:
# Install required packages in Colab
import os
import sys

# Check if running in Colab
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    !pip install ultralytics
    !pip install roboflow
    print("Packages installed successfully!")
else:
    print("Not running in Colab - assuming packages are already installed")

In [ ]:
import os
import sys
import logging
from pathlib import Path
from typing import Dict, Optional

import torch
import yaml
from ultralytics import YOLO
import matplotlib.pyplot as plt
import numpy as np
import cv2
from IPython.display import display, Image as IPImage
import pandas as pd

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")
    print(f"CUDA memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")

## 2. Configuration and Setup

In [ ]:
# Training configuration
CONFIG = {
    'model': {
        'architecture': 'yolov8n',
        'pretrained': True
    },
    'training': {
        'epochs': 80,
        'batch_size': 16,
        'learning_rate': 0.001,
        'patience': 50,
        'optimizer': 'AdamW',
        'momentum': 0.937,
        'weight_decay': 0.0005,
        'warmup_epochs': 3,
        'augmentation': {
            'hsv_h': 0.015,
            'hsv_s': 0.7,
            'hsv_v': 0.4,
            'degrees': 10.0,
            'translate': 0.2,
            'scale': 0.5,
            'shear': 2.0,
            'perspective': 0.0002,
            'flipud': 0.5,
            'fliplr': 0.5,
            'mosaic': 1.0,
            'mixup': 0.15,
            'copy_paste': 0.3
        }
    },
    'validation': {
        'conf_threshold': 0.25,
        'iou_threshold': 0.5,
        'max_det': 1000
    }
}

# Dataset configuration
DATASET_CONFIG = {
    'classes': ['FireExtinguisher', 'ToolBox', 'OxygenTank'],
    'nc': 3,
    'train': 'data/train/images',
    'val': 'data/val/images',
    'test': 'data/test/images'
}

print("Configuration loaded successfully!")
print(f"Classes: {DATASET_CONFIG['classes']}")
print(f"Training epochs: {CONFIG['training']['epochs']}")
print(f"Batch size: {CONFIG['training']['batch_size']}")

## 3. Dataset Preparation

In [ ]:
# Setup logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler(sys.stdout)
    ]
)
logger = logging.getLogger(__name__)

def get_device():
    """Get the best available device."""
    if torch.cuda.is_available():
        device = torch.device('cuda')
        logger.info(f"Using GPU: {torch.cuda.get_device_name(0)}")
    else:
        device = torch.device('cpu')
        logger.info("Using CPU")
    return device

device = get_device()

In [ ]:
# Create dataset YAML file
def create_dataset_yaml(config_dict, save_path='dataset.yaml'):
    """Create dataset YAML configuration file."""
    with open(save_path, 'w') as f:
        yaml.dump(config_dict, f, default_flow_style=False)
    logger.info(f"Dataset YAML created: {save_path}")
    return save_path

# Create the dataset YAML
dataset_yaml_path = create_dataset_yaml(DATASET_CONFIG)

# Display the YAML content
with open(dataset_yaml_path, 'r') as f:
    print("Dataset YAML content:")
    print(f.read())

## 4. Data Analysis and Visualization

In [ ]:
def analyze_dataset(data_path='data'):
    """Analyze the dataset structure and statistics."""
    data_path = Path(data_path)
    
    splits = ['train', 'val', 'test']
    stats = {}
    
    for split in splits:
        images_path = data_path / split / 'images'
        labels_path = data_path / split / 'labels'
        
        if images_path.exists():
            image_files = list(images_path.glob('*.png')) + list(images_path.glob('*.jpg'))
            stats[split] = {
                'images': len(image_files),
                'labels': len(list(labels_path.glob('*.txt'))) if labels_path.exists() else 0
            }
        else:
            stats[split] = {'images': 0, 'labels': 0}
    
    # Display statistics
    print("Dataset Statistics:")
    print("="*50)
    for split, data in stats.items():
        print(f"{split.capitalize()}: {data['images']} images, {data['labels']} labels")
    
    total_images = sum(data['images'] for data in stats.values())
    total_labels = sum(data['labels'] for data in stats.values())
    print(f"Total: {total_images} images, {total_labels} labels")
    
    return stats

# Analyze dataset if it exists
if os.path.exists('data'):
    dataset_stats = analyze_dataset()
else:
    print("Dataset not found. Please upload your dataset to the 'data' directory.")
    print("Expected structure:")
    print("data/")
    print("├── train/")
    print("│   ├── images/")
    print("│   └── labels/")
    print("├── val/")
    print("│   ├── images/")
    print("│   └── labels/")
    print("└── test/")
    print("    ├── images/")
    print("    └── labels/")

In [ ]:
def plot_class_distribution(labels_dir, class_names):
    """Plot class distribution from label files."""
    labels_dir = Path(labels_dir)
    class_counts = {i: 0 for i in range(len(class_names))}
    
    for label_file in labels_dir.glob('*.txt'):
        try:
            with open(label_file, 'r') as f:
                for line in f:
                    if line.strip():
                        class_id = int(line.split()[0])
                        if class_id in class_counts:
                            class_counts[class_id] += 1
        except Exception as e:
            continue
    
    # Plot distribution
    plt.figure(figsize=(10, 6))
    classes = [class_names[i] for i in range(len(class_names))]
    counts = [class_counts[i] for i in range(len(class_names))]
    
    plt.bar(classes, counts, color=['#FF6B6B', '#4ECDC4', '#45B7D1'])
    plt.title('Class Distribution in Training Set', fontsize=16, fontweight='bold')
    plt.xlabel('Classes', fontsize=12)
    plt.ylabel('Number of Instances', fontsize=12)
    plt.xticks(rotation=45)
    
    # Add value labels on bars
    for i, v in enumerate(counts):
        plt.text(i, v + max(counts) * 0.01, str(v), ha='center', va='bottom', 
                 fontweight='bold', fontsize=11)
    
    plt.tight_layout()
    plt.grid(axis='y', alpha=0.3)
    plt.show()
    
    return class_counts

# Plot class distribution if training labels exist
if os.path.exists('data/train/labels'):
    class_distribution = plot_class_distribution('data/train/labels', DATASET_CONFIG['classes'])
    print(f"Class distribution: {class_distribution}")
else:
    print("Training labels not found. Skipping class distribution plot.")

## 5. Model Training

In [ ]:
def setup_training_parameters(config):
    """Setup training parameters from configuration."""
    
    device_str = 'cuda' if torch.cuda.is_available() else 'cpu'
    
    train_params = {
        'data': 'dataset.yaml',
        'epochs': config['training']['epochs'],
        'batch': config['training']['batch_size'],
        'lr0': config['training']['learning_rate'],
        'imgsz': 640,
        'device': device_str,
        'workers': 8 if device_str == 'cuda' else 4,
        'project': 'runs/train',
        'name': 'industrial_safety_detection',
        'save_period': 10,
        'patience': config['training']['patience'],
        'optimizer': config['training']['optimizer'],
        'momentum': config['training']['momentum'],
        'weight_decay': config['training']['weight_decay'],
        'warmup_epochs': config['training']['warmup_epochs'],
        'verbose': True,
        'seed': 42,
        'deterministic': True,
        'single_cls': False,
        'rect': False,
        'cos_lr': True,
        'close_mosaic': 10,
        'amp': True,
        'fraction': 1.0,
        'profile': False,
        'freeze': None,
        'multi_scale': False,
        'overlap_mask': True,
        'mask_ratio': 4,
        'dropout': 0.0,
        'val': True,
        'plots': True,
        'save': True,
        'save_json': False,
        'save_hybrid': False,
        'conf': config['validation']['conf_threshold'],
        'iou': config['validation']['iou_threshold'],
        'max_det': config['validation']['max_det'],
        'half': False,
        'dnn': False,
        'exist_ok': True,
    }
    
    # Add augmentation parameters
    aug_config = config['training']['augmentation']
    train_params.update({
        'hsv_h': aug_config['hsv_h'],
        'hsv_s': aug_config['hsv_s'],
        'hsv_v': aug_config['hsv_v'],
        'degrees': aug_config['degrees'],
        'translate': aug_config['translate'],
        'scale': aug_config['scale'],
        'shear': aug_config['shear'],
        'perspective': aug_config['perspective'],
        'flipud': aug_config['flipud'],
        'fliplr': aug_config['fliplr'],
        'mosaic': aug_config['mosaic'],
        'mixup': aug_config['mixup'],
        'copy_paste': aug_config['copy_paste'],
    })
    
    return train_params

# Setup training parameters
train_params = setup_training_parameters(CONFIG)

print("Training Parameters:")
print("="*50)
for key, value in train_params.items():
    if key in ['hsv_h', 'hsv_s', 'hsv_v', 'degrees', 'translate', 'scale', 'shear', 'perspective']:
        continue  # Skip augmentation params for brevity
    print(f"{key}: {value}")
print("\nAugmentation enabled with optimized parameters")

In [ ]:
# Load and initialize the model
model_name = f"{CONFIG['model']['architecture']}.pt"
logger.info(f"Loading model: {model_name}")

model = YOLO(model_name)

# Model information
print(f"Model: {model_name}")
print(f"Parameters: {sum(p.numel() for p in model.model.parameters()):,}")
print(f"Model summary:")
model.info()

In [ ]:
# Start training
logger.info("Starting training...")
print("\n" + "="*60)
print("STARTING TRAINING")
print("="*60)

try:
    results = model.train(**train_params)
    logger.info("Training completed successfully!")
    
    print("\n" + "="*60)
    print("TRAINING COMPLETED SUCCESSFULLY!")
    print("="*60)
    
except Exception as e:
    logger.error(f"Training failed: {e}")
    print(f"Training failed with error: {e}")
    raise

## 6. Training Results Analysis

In [ ]:
# Display training results
results_dir = Path('runs/train/industrial_safety_detection')

if results_dir.exists():
    print(f"Results saved to: {results_dir}")
    print("\nTraining artifacts:")
    
    # List key files
    key_files = [
        'results.png',
        'confusion_matrix.png', 
        'train_batch0.jpg',
        'val_batch0_pred.jpg',
        'weights/best.pt',
        'weights/last.pt'
    ]
    
    for file in key_files:
        file_path = results_dir / file
        if file_path.exists():
            print(f"  ✓ {file}")
        else:
            print(f"  ✗ {file} (not found)")
else:
    print("Results directory not found. Training may not have completed successfully.")

In [ ]:
# Display training curves
results_img_path = results_dir / 'results.png'
if results_img_path.exists():
    print("Training Results:")
    display(IPImage(str(results_img_path)))
else:
    print("Training results image not found.")

In [ ]:
# Display confusion matrix
confusion_matrix_path = results_dir / 'confusion_matrix.png'
if confusion_matrix_path.exists():
    print("Confusion Matrix:")
    display(IPImage(str(confusion_matrix_path)))
else:
    print("Confusion matrix not found.")

## 7. Model Validation

In [ ]:
# Load best model and run validation
best_model_path = results_dir / 'weights' / 'best.pt'
if best_model_path.exists():
    logger.info("Loading best model for validation...")
    best_model = YOLO(str(best_model_path))
    
    # Run validation
    logger.info("Running final validation...")
    val_results = best_model.val()
    
    print("\n" + "="*50)
    print("FINAL VALIDATION RESULTS")
    print("="*50)
    print(f"mAP@0.5: {val_results.box.map50:.4f}")
    print(f"mAP@0.5:0.95: {val_results.box.map:.4f}")
    print(f"Precision: {val_results.box.mp:.4f}")
    print(f"Recall: {val_results.box.mr:.4f}")
    print("="*50)
    
else:
    print("Best model not found. Training may not have completed successfully.")

## 8. Sample Predictions

In [ ]:
# Display sample validation predictions
val_pred_path = results_dir / 'val_batch0_pred.jpg'
if val_pred_path.exists():
    print("Sample Validation Predictions:")
    display(IPImage(str(val_pred_path)))
else:
    print("Validation predictions image not found.")

In [ ]:
# Test on a sample image if test data exists
test_images_dir = Path('data/test/images')
if test_images_dir.exists() and best_model_path.exists():
    test_images = list(test_images_dir.glob('*.png'))[:3]  # Take first 3 images
    
    if test_images:
        print("Sample Test Predictions:")
        
        for i, img_path in enumerate(test_images):
            print(f"\nImage {i+1}: {img_path.name}")
            
            # Run prediction
            results = best_model.predict(str(img_path), conf=0.25, save=True, project='runs/predict', name=f'test_sample_{i}')
            
            # Display results
            for result in results:
                if len(result.boxes) > 0:
                    print(f"  Detected {len(result.boxes)} objects")
                    for box in result.boxes:
                        class_id = int(box.cls)
                        confidence = float(box.conf)
                        class_name = DATASET_CONFIG['classes'][class_id]
                        print(f"    {class_name}: {confidence:.3f}")
                else:
                    print("  No objects detected")
    else:
        print("No test images found.")
else:
    print("Test data or trained model not available for sample predictions.")

## 9. Export Model

In [ ]:
# Export model to different formats
if best_model_path.exists():
    logger.info("Exporting model to different formats...")
    
    try:
        # Export to ONNX
        best_model.export(format='onnx', opset=11)
        print("✓ Model exported to ONNX format")
        
        # Export to TensorRT (if available)
        if torch.cuda.is_available():
            try:
                best_model.export(format='engine')
                print("✓ Model exported to TensorRT format")
            except Exception as e:
                print(f"⚠ TensorRT export failed: {e}")
        
        # Export to CoreML (for iOS)
        try:
            best_model.export(format='coreml')
            print("✓ Model exported to CoreML format")
        except Exception as e:
            print(f"⚠ CoreML export failed: {e}")
            
    except Exception as e:
        print(f"Export failed: {e}")
else:
    print("Best model not found. Cannot export.")

## 10. Summary and Next Steps

In [ ]:
print("\n" + "="*60)
print("TRAINING SUMMARY")
print("="*60)
print(f"Model: {CONFIG['model']['architecture']}")
print(f"Classes: {', '.join(DATASET_CONFIG['classes'])}")
print(f"Training epochs: {CONFIG['training']['epochs']}")
print(f"Batch size: {CONFIG['training']['batch_size']}")
print(f"Optimizer: {CONFIG['training']['optimizer']}")
print(f"Device: {'GPU' if torch.cuda.is_available() else 'CPU'}")

if best_model_path.exists():
    print(f"\nModel saved to: {best_model_path}")
    print(f"Results directory: {results_dir}")
    
print("\nNext Steps:")
print("1. Review training curves and validation results")
print("2. Test the model on new images using inference notebook")
print("3. Fine-tune hyperparameters if needed")
print("4. Deploy the model for production use")
print("="*60)